In [ ]:
"""
Count row of each file, grouped by axis, and output to csv.
"""

import tkinter as tk

from pathlib import Path
from tkinter import filedialog

import pandas as pd
from IPython import get_ipython

get_ipython().run_line_magic("gui", "tk")
root = tk.Tk()
root.withdraw()
root.call("wm", "attributes", ".", "-topmost", True)


def prepare_df(directory: Path) -> pd.DataFrame:
    df = pd.concat(
        [
            pd.read_csv(directory / file).assign(file=file.name)
            for file in directory.iterdir()
            if file.suffix == ".csv" and not file.name == "output.csv"
        ]
    ).rename(columns={" ": "cell"})

    df["file"] = df["file"].astype("category")
    return df[[(u := df.columns.tolist())[-1]] + u[:-1]]


directory = Path(filedialog.askdirectory(title="Choose folder directory"))

axis = input(
    "Axis to segment by, X or Y need to be capitalized. Left blank if not going to segment. "
)

if axis == "":
    segment = 0
else:
    segment = int(input("Enter the `width` of each layer. "))


df = prepare_df(directory)

if segment == 0:
    df["layer"] = 0
else:
    vals = df[axis] - df[axis].min()
    df["layer"] = (vals // segment).astype(int).astype("category")

df = df.groupby(["file", "layer"]).size().reset_index()
df = df.pivot(index="file", columns="layer", values=0)

df.to_csv(directory / "output.csv")

print("Done!")